In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
filepath = "data/A - AmbulanceCalls.xlsx"

ambulance_train_df = pd.read_excel(filepath, sheet_name='train')
ambulance_test_df = pd.read_excel(filepath, sheet_name='test')

In [ ]:
with pd.option_context("display.max_rows", None):
    print(ambulance_train_df)

In [ ]:
ambulance_test_df

In [ ]:
print(f"Train shape: {ambulance_train_df.shape}")
print(f"Test shape: {ambulance_test_df.shape}")

### Exploratory Data Analysis

1.   Does the data contain any trends? How do you know?
2.   Does the data contain seasonality? How do you know?
    *Cant use: Dickey-Fuller test, Augmented Dickey-Fuller test, KPSS test

3. Were there any major outliers (i.e., large upward or downward spikes)? Should they be included in the forecasting analysis or not? Why?

4. Through your data analysis, determine what type of forecasting methods would be appropriate for use in forecasting future values for the selected dataset.











#### 1. Trend Analysis

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(ambulance_train_df["date"], ambulance_train_df["calls"])
plt.title("Train Data: Ambulance Calls")
plt.xlabel("Date")
plt.ylabel("# of Calls")
plt.xticks(rotation=45)
plt.plot()

In [ ]:
ma_windows = [5,26,52]

fig, ax = plt.subplots(figsize=(15,10))

for window in ma_windows:
    ma_series = ambulance_train_df['calls'].rolling(window=window, center=True).mean()
    ax.plot(ambulance_train_df["date"], ma_series, label=f"MA Window Size: {window} weeks")

ax.set_title("Train Data: Smoothed Ambulance Calls w MA")
ax.set_xlabel("Date")
ax.set_ylabel("# of Calls")
plt.legend()
plt.show()

When we plot the original ambulance call data, we observe clear cycles, but it is difficult to identify any longer term trends. We do notice that the third peak appears slightly broader than the first two suggesting a possible trend.

To better visualize the data and reduce noise, we apply moving averages (MA). Considering the cycles in the data, we looked at typical time frames: each month has approximately 3–5 data points (weeks), while each year has roughly 51–52 data points.

Using these as our sliding windows, we apply MA with windows of 5 (roughly a month) and 52 (roughly a year). We observe that as the MA window increases, the fluctuations smooth out and the underlying trend becomes more apparent. In particular, the yearly MA reveals a gradual upward trend, confirming that a long-term trend exists in the data once seasonality is smoothed out.

#### 2. Seasonality

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(ambulance_train_df['calls'], lags=104) # Compute autocorr for evry lag up to 104
plt.title("Autocorrelation of Ambulance Calls")
plt.xlabel("Lag (by num of data points)")
plt.ylabel("Autocorrelation")
plt.show()

When plotting the raw training data, we can clearly identify cycles, notably: the first cycle from 2020-01 to 2021-01, the second cycle from 2021-01 to 2022-05, and the last cycle from 2022-05 to 2023-05.

To further verify the presence of seasonality, we plot the autocorrelation function (ACF) of the data. From the ACF, we observe that the series exhibits weak autocorrelation with its lagged values, with the strongest correlation of approximately 0.3 at a lag of ~53, which corresponds to roughly one year’s worth of data points, consistent with the cycles identified in the time series. The weak autocorrelation can be the result of the noisy data so to further explore we can retry the acf on the smmothed data (via MA) to find stronger correlation

Also a decay suggests non-stationary (ex: trend)

In [ ]:
for window in ma_windows:
    ma_series = ambulance_train_df['calls'].rolling(window=window, center=True).mean().dropna()
    plot_acf(ma_series, lags=104) # Compute autocorr for evry lag up to 104
    plt.title(f"Autocorrelation of Ambulance Calls w/ MA Window Size:{window}")
    plt.xlabel("Lag (by num of data points)")
    plt.ylabel("Autocorrelation")
    plt.show()

#### 3. Outlier Analysis

#### Training Data

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(ambulance_train_df['calls'], model='additive', period=52)
residuals = decomposition.resid.dropna()
residuals.plot()

In [ ]:
# Std and mean
resid_mean = residuals.mean()
resid_std = residuals.std()
lower_bound = resid_mean - 3 * resid_std
upper_bound = resid_mean + 3 * resid_std

outliers = residuals[(residuals < lower_bound) | (residuals > upper_bound)]
print(f"Outliers found through stdeva: {outliers}")

In [ ]:
# Quantiles
Q1 = residuals.quantile(0.25)
Q3 = residuals.quantile(0.75)
IQR = Q3 - Q1

robust_lower = Q1 - 1.5 * IQR
robust_upper = Q3 + 1.5 * IQR

outliers_iqr = residuals[(residuals < robust_lower) | (residuals > robust_upper)]

print("Outliers found using Robust IQR:")
print(outliers_iqr)

The difference between the two methods highlights a statistical property known as "masking." The stdeva method failed to detect any outliers because the extreme spike on September 19 2022 was large enough to inflate the entire datasets stdeva. This inflated value widened the upper boundary significantly that the outlier was effectively hidden within the expanded definition of "normal." 

The IQR method is robust to extreme values because it constructs boundaries based on the middle 50% of the data. By ignoring the extremes during calculation, the IQR method correctly identified the residual of approximately 207 as a statistically significant anomaly that the standard deviation method missed.

### Forecasting

#### 1. Triple Exponential Smoothing

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

model = ExponentialSmoothing(
    ambulance_train_df['calls'], 
    trend='add', 
    seasonal='add', 
    seasonal_periods=52
)

fitted_model = model.fit()

forecast_steps = len(ambulance_test_df)
forecast = fitted_model.forecast(steps=forecast_steps)
forecast.index = ambulance_test_df['date']

plt.figure(figsize=(12, 5))
plt.plot(ambulance_train_df['date'], ambulance_train_df['calls'], label='Actual Data', color='blue')
plt.plot(ambulance_train_df['date'], fitted_model.fittedvalues, label='Model Fit (Training)', color='orange', alpha=0.7)
plt.plot(forecast.index, forecast, label='Forecast (Next 6 Months)', color='red', linestyle='--')
plt.title('Triple Exponential Smoothing (Winters) Forecast')
plt.xlabel('Date')
plt.ylabel('Ambulance Calls')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print(fitted_model.params)

The TES Winters model optimized to a smoothing level of 0.37, indicating moderate responsiveness to recent changes. Notably, both trend and seasonal smoothing were 0.0, indicating that the underlying seasonal structure (Winter Lows/Summer Highs) is highly stable and does not evolve significantly over time. The seasonal factors confirm this, showing that peak summer weeks consistently.

In [ ]:
from sklearn.metrics import mean_absolute_error

mape = mean_absolute_error(ambulance_train_df['calls'], fitted_model.fittedvalues)
print("MAE on fitted training data:", mape)

In [ ]:
error_df = pd.DataFrame(index=['Training', 'Test'])
error_df.loc["Training", "TES"] = 63.00824924211579

##### Sensitivty Analysis 

In [ ]:
def get_forecast(alpha, beta, gamma):
    model = ExponentialSmoothing(
        ambulance_train_df['calls'], 
        trend='add', 
        seasonal='mul', 
        seasonal_periods=52
    )

    fitted_model = model.fit(
        smoothing_level=alpha, 
        smoothing_trend=beta, 
        smoothing_seasonal=gamma, 
        optimized=False
    )
    
    return fitted_model.forecast(30)

base_alpha = 0.344
base_beta  = 0.0
base_gamma = 0.0

# Scenario 1: Base Case 
forecast_base = get_forecast(base_alpha, base_beta, base_gamma)

# Scenario 2: Sensitivity to Level (Alpha +- 10%)
forecast_alpha_high = get_forecast(base_alpha * 1.5, base_beta, base_gamma)
forecast_alpha_low  = get_forecast(base_alpha * 0.5, base_beta, base_gamma)

# Scenario 3: Sensitivity to Trend (Force Beta = 0.05)
forecast_beta_forced = get_forecast(base_alpha, 0.05, base_gamma)

# Scenario 4: Sensitivity to Seasonality (Force Gamma = 0.05)
forecast_gamma_forced = get_forecast(base_alpha, base_beta, 0.05)

# Impact
diff_alpha = np.mean(np.abs(forecast_base - forecast_alpha_high))
diff_beta  = np.mean(np.abs(forecast_base - forecast_beta_forced))
diff_gamma = np.mean(np.abs(forecast_base - forecast_gamma_forced))

print("Sensitivity Report (Avg Change in 30-Week Forecast):")
print(f"1. Adjusting Alpha by 50%:   {diff_alpha:.2f} calls/week")
print(f"2. Forcing Trend (Beta=0.05): {diff_beta:.2f} calls/week")
print(f"3. Forcing Season (Gamma=0.05): {diff_gamma:.2f} calls/week")


plt.figure(figsize=(12, 6))
plt.plot(ambulance_train_df['date'].iloc[-52:], ambulance_train_df['calls'].iloc[-52:], label='Historical Data (Last 52 Weeks)', color='gray', alpha=0.5)
plt.plot(ambulance_test_df['date'], forecast_base, label='Base Model (Beta=0, Gamma=0)', color='black', linewidth=2.5)
plt.plot(ambulance_test_df['date'], forecast_alpha_high, label='Alpha +50%', color='blue', linestyle='--')
plt.plot(ambulance_test_df['date'], forecast_alpha_low, label='Alpha -50%', color='pink', linestyle='--')
plt.plot(ambulance_test_df['date'], forecast_beta_forced, label='Force Trend (Beta=0.05)', color='red', linestyle=':')
plt.plot(ambulance_test_df['date'], forecast_gamma_forced, label='Force Season (Gamma=0.05)', color='green', linestyle=':')
plt.title("Sensitivity Analysis: How Parameters Impact the Forecast")
plt.xlabel("Date")
plt.ylabel("Calls")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

#### 2. ARIMA

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# First Difference -> d=1
diff_series = ambulance_train_df['calls'].diff().dropna()
plt.figure(figsize=(12,4))
plt.plot(diff_series)
plt.title("First Differenced Series (d=1)")
plt.grid(True)
plt.show()

# PACF -> p
fig, ax = plt.subplots(figsize=(12,5))
plot_pacf(diff_series, lags=20, method='ywm', ax=ax)
ax.set_xlim(0, 20)
ax.set_xticks(np.arange(0, 21, 1))
ax.grid(True, axis='both')
ax.set_xlabel("Lag")
ax.set_ylabel("Partial Autocorrelation")
ax.set_title("PACF Plot (Use this to choose p)")
plt.show()

# ACF -> q
fig, ax = plt.subplots(figsize=(12,5))
plot_acf(diff_series, lags=20, ax=ax)
ax.set_xlim(0, 20)
ax.set_xticks(np.arange(0, 21, 1))
ax.grid(True, axis='both')
ax.set_xlabel("Lag")
ax.set_ylabel("Autocorrelation")
ax.set_title("ACF Plot (Use this to choose q)")
plt.show()

We choose p = 2 and q = 1 and d = 1

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arima_model = ARIMA(ambulance_train_df['calls'], order=(2, 1, 1))
arima_fitted = arima_model.fit()
forecast_arima = arima_fitted.forecast(steps=30)

plt.figure(figsize=(12,5))
plt.plot(ambulance_train_df['calls'], label='Observed', color="blue")
plt.plot(arima_fitted.fittedvalues, label='Fitted', color="orange")
plt.plot(forecast_arima, label='Forecast', color="red", linestyle= '--')
plt.legend()
plt.show()

print(arima_fitted.summary())

In [ ]:
mae = mean_absolute_error(ambulance_train_df['calls'], arima_fitted.fittedvalues)
print("MAE:", mae)

In [ ]:
error_df.loc["Training", "ARIMA"] = 79.65246552845532

#### Sensitivty Analysis

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

def get_arima_forecast(order):
    model = ARIMA(ambulance_train_df['calls'], order=order)
    res = model.fit()
    return res.forecast(steps=30)

# Base Case: Your winner
base_order = (2, 1, 1)
forecast_base = get_arima_forecast(base_order)

# Scenario 1: Sensitivity to AR Term 
forecast_p_low = get_arima_forecast((1, 1, 1))
forecast_p_high = get_arima_forecast((3, 1, 1))

# Scenario 2: Sensitivity to MA Term 
forecast_q_low = get_arima_forecast((2, 1, 0))
forecast_q_high = get_arima_forecast((2, 1, 2))

diff_p_low  = np.mean(np.abs(forecast_base - forecast_p_low))
diff_p_high = np.mean(np.abs(forecast_base - forecast_p_high))
diff_q_low  = np.mean(np.abs(forecast_base - forecast_q_low))
diff_q_high = np.mean(np.abs(forecast_base - forecast_q_high))

print("ARIMA Sensitivity Report (Avg Change in 30-Week Forecast):")
print(f"1. Reducing AR (p=1):    {diff_p_low:.2f} calls/week")
print(f"2. Increasing AR (p=3):  {diff_p_high:.2f} calls/week")
print(f"3. Removing MA (q=0):    {diff_q_low:.2f} calls/week")
print(f"2. Increasing MA (q=2):    {diff_q_high:.2f} calls/week")


plt.figure(figsize=(12, 6))
plt.plot(ambulance_train_df['date'].iloc[-52:], ambulance_train_df['calls'].iloc[-52:], label='Historical Data (Last 52 Weeks)', color='gray', alpha=0.5)
plt.plot(ambulance_test_df['date'], forecast_base, label='Base Model (2,1,1)', color='black', linewidth=2.5)
plt.plot(ambulance_test_df['date'], forecast_p_low, label='Reduce AR (1,1,1)', color='blue', linestyle='--')
plt.plot(ambulance_test_df['date'], forecast_p_high, label='Increase AR (3,1,1)', color='pink', linestyle=':')
plt.plot(ambulance_test_df['date'], forecast_q_low, label='Remove MA (2,1,0)', color='red', linestyle=':')
plt.plot(ambulance_test_df['date'], forecast_q_high, label='Remove MA (2,1,2)', color='green', linestyle=':')
plt.title("ARIMA Sensitivity Analysis: Structural Changes (Model Orders)")
plt.xlabel("Date")
plt.ylabel("Calls")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

#### Grid Search with SARIMAX

In [ ]:
import pmdarima as pm

ambulance_train_df['date'] = pd.to_datetime(ambulance_train_df['date'])
ambulance_test_df['date'] = pd.to_datetime(ambulance_test_df['date']) 

arima_model = pm.auto_arima(
    ambulance_train_df['calls'],
    seasonal=True,
    m=52,
    d=1,
    D=1,
    trace=True,
    error_action="ignore",
    suppress_warnings=True,
    stepwise=True
)

print(arima_model.summary)

forecast_arima = arima_model.predict(n_periods=len(ambulance_test_df))

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# order=(0, 1, 1)
# seasonal_order=(2, 1, 0, 52)
sarima_model = SARIMAX(
    ambulance_train_df['calls'],
    order=(0, 1, 1),
    seasonal_order=(2, 1, 0, 52)
)

results = sarima_model.fit()

forecast_arima = results.get_forecast(steps=len(ambulance_test_df))
predicted_mean = forecast_arima.predicted_mean
predicted_mean.index = ambulance_test_df['date']
fitted_values = results.fittedvalues
fitted_values.index = ambulance_train_df['date']

plt.figure(figsize=(12, 6))
plt.plot(ambulance_train_df['date'], ambulance_train_df['calls'], label='History', color='blue')
plt.plot(fitted_values.index, fitted_values, label='Model Fit (Training)', color='orange')
plt.plot(predicted_mean.index, predicted_mean, label='SARIMAX Forecast', color='red', linestyle='--')
plt.title('Final SARIMAX Forecast: (0,1,1)(2,1,0)[52]')
plt.legend()
plt.show()

In [ ]:
mae = mean_absolute_error(ambulance_train_df['calls'], fitted_values)
print("MAE on fitted training data:", mae)

In [ ]:
error_df.loc["Training", "SARIMAX"] = 93.8673065514869

###  Comparison to Actual Data

MAE was selected as the primary metric because it provides a tangible value in "calls per week". For a system like EMS, where resource allocation (# of ambulances) is based on discrete units of demand, MAE offers the most actionable insight into the buffer capacity required to maintain service levels.

#### Triple Exponential Smoothing

In [ ]:
plt.figure(figsize=(12, 6))

plt.plot(ambulance_test_df['date'], ambulance_test_df['calls'], label='Actual Demand', color='blue')
plt.plot(forecast.index, forecast, label='Forecast (Next 1 Year)', color='red', linestyle='--')
plt.title("Actual Demand vs Forecast (Triple Exponential Smoothing)")
plt.xlabel("Date")
plt.ylabel("# of Calls")
plt.legend()
plt.show()

In [ ]:
mae = mean_absolute_error(ambulance_test_df['calls'], forecast.values)
print("MAE on test data:", mae)

In [ ]:
error_df.loc["Test", "TES"] = 139.68399079764032

#### ARIMA

In [ ]:
forecast_values = arima_fitted.forecast(steps=len(ambulance_test_df))
forecast_series = pd.Series(forecast_values.values, index=ambulance_test_df['date'])

plt.figure(figsize=(12, 6))
plt.plot(ambulance_test_df['date'], ambulance_test_df['calls'], label='Actual Demand', color='blue')
plt.plot(forecast_series.index, forecast_series.values, label='SARIMA Forecast', color='red', linestyle='--')
plt.title("Actual Demand vs Forecast (SARIMAX)")
plt.xlabel("Date")
plt.ylabel("# of Calls")
plt.legend()
plt.show()

In [ ]:
mae = mean_absolute_error(ambulance_test_df['calls'], forecast_series.values)
print("MAE on test data:", mae)

In [ ]:
error_df.loc["Test", "ARIMA"] = 78.67921835901419

### SARIMAX

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ambulance_test_df['date'], ambulance_test_df['calls'], label='Actual Demand', color='blue')
plt.plot(predicted_mean.index, predicted_mean, label='SARIMA Forecast', color='red', linestyle='--')
plt.title("Actual Demand vs Forecast (SARIMAX)")
plt.xlabel("Date")
plt.ylabel("# of Calls")
plt.legend()
plt.show()

In [ ]:
mae = mean_absolute_error(ambulance_test_df['calls'], predicted_mean.values)
print("MAE on test data:", mae)

In [ ]:
error_df.loc["Test", "SARIMAX"] = 184.0077317600946

In [ ]:
error_df

#### 1. Which forecasting method was better?

ARIMA resulted in a lower MAE with 78.68 compared to TES with a MAE of 139.68 and SARIMAX with a MAE of 184.01.

- While the Triple Exponential Smoothing (TES) model fit the training data more closely (Training MAE: 63.01), its error more than doubled in the test phase (Test MAE: 139.68). This indicates that TES overfitted to the specific of the past
- The ARIMA model demonstrated very good stability, with its test error (78.68) being nearly identical to its training error (79.65). By producing a more conservative straight-line forecast, it ignored non-repeating noise and captured the true underlying trend better than the more spiky seasonal models

#### 2. Any outliers that impacted forecast accuracy?

In [ ]:
# We use the forecast variable from ARIMA
test_residuals = ambulance_test_df['calls'] - forecast_series.values

Q1_t = test_residuals.quantile(0.25)
Q3_t = test_residuals.quantile(0.75)
IQR_t = Q3_t - Q1_t

lower_t = Q1_t - 1.5 * IQR_t
upper_t = Q3_t + 1.5 * IQR_t

test_outliers = ambulance_test_df[(test_residuals < lower_t) | (test_residuals > upper_t)]

print("Outliers found in the 30-week Test Data:")
print(test_outliers)

No outliers were found in the 30-week test period using our ARIMA forecast and IQR calculations. This indicates that during the 30 weeks test data, the ambulance call volume followed our ARIMA model's predicted patterns without any extreme or unexplained shocks.

#### 3. Which method would your recommend using for your particular use case and why?

I recommend using the ARIMA model for this ambulance call use case. or Industrial Engineering and resource management (staffing ambulances), a model that generalizes well to new data is critical. The ARIMA model provides a reliable baseline (MAE ~79) regardless of whether it is looking at the past or the future. Relying on the TES model would be riskier, as its performance proved to be highly inconsistent when faced with new, unseen data.

#### 4. What are the strengths and weaknesses of models? What external info. would have helped improve forecast accuracy?



ARIMA: 
- PROS: High stability and generalization, less likely to overfit, works well with smaller test sets
- CONS: Produces a flat trend that may miss subtle seasonal oscillations

TES:
- PROS: Very good at capturing complex seasonal waves qualatatively for historical data.
- CONS: Highly prone to overfitting as seen when forecsting and calculating error with test set. Sensitive to historical spikes which it tries to replicate.
